In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API 
(for problem finding)

In [2]:
import requests
import json
import os
from dotenv import load_dotenv

# Make sure you have an .env file with the relevant API keys from OpenAI and Mergeflow 
# Infos under following link: https://blog.bitsrc.io/a-gentle-introduction-to-env-files-9ad424cc5ff4

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
username = 'leonard' # replace by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
query = 'factory automation' # topic of the search
dataset = '&sp=2655' # access dataset from mergeflow, in this case scientific publications
rows = str(5) # first 5 docs only

url = f'https://mergeflow.net/api/v1/{username}/search/getContent?q={query}{dataset}&rows={rows}'

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Problem Finding

In [5]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

       For the problem you found, please only output the following: 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 250-300 words)
        - "User group": Which user group is impacted by this problem the most (please write 20-50 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 80-100 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem:

"Title": Adaptive Energy Management

"Explanation": In the realm of factory automation, the integration of deep reinforcement learning (DRL) and advanced simulation technologies marks a significant leap forward. However, an emerging challenge lies in the optimization of energy consumption amidst these advancements. With factories increasingly relying on automated systems powered by sophisticated algorithms and simulations, the demand for energy is escalating, leading to higher operational costs and a larger carbon footprint. The problem intensifies as these technologies become more integrated into daily operations, requiring continuous power for complex computations and machinery operation. Moreover, the variability in energy prices and the push for sustainability demand a more dynamic approach to energy management. Traditional static energy usage strategies are becoming obsolete, unable to adapt to the fluctuating nature of energy markets or to leverage periods of low demand

Problem:

- "Title": Adaptive Cybersecurity Threats

- "Explanation": In the realm of factory automation, the integration of intelligent sensing, control, and optimization networks has significantly enhanced operational efficiency, reliability, and performance. However, this technological evolution brings forth the adjacent possible problem of adaptive cybersecurity threats. As factories become more interconnected and reliant on cyber-physical systems, they become more vulnerable to sophisticated cyber-attacks that can adapt to defensive measures in real-time. These threats are not just limited to data breaches but can extend to physical disruptions in manufacturing processes, leading to significant financial losses, endangering worker safety, and damaging brand reputation. The emergence of these adaptive threats is a direct consequence of the advanced algorithms and machine learning models embedded within factory automation systems, which, while optimizing operations, also open new ve

In [6]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria.
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)
        
    Be very stringent with your ratings:
        1: Does not meet expectations
        2: Meets expectations adequately
        3: Exceeds expectations significantly
    
    A rating of 3 should be reserved for exceptional performance in the given criterion.

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 50 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

**Problem: Adaptive Energy Management**

- Size of user group: 3
- Relevancy of pain point: 3
- How specific is the problem: 3

**Overall score: 9/9**

Justification: Adaptive Energy Management is highly relevant and specific to factory operations managers, targeting a large user group seeking to optimize energy consumption for cost savings and sustainability.

---

Problem rating:

**Problem: Adaptive Automation Learning**

- **Size of user group**: 3  
- **Relevancy of pain point**: 3  
- **How specific is the problem?**: 3  

**Overall Score**: 9/9

**Justification**: The problem of adaptive automation learning is highly relevant as it directly addresses the critical need for continuous skill development among a large group of factory workers and engineers in the rapidly evolving field of factory automation.

---

Problem rating:

Title: Adaptive Safety Compliance

- Size of user group: 3  
Justification: The user group includes a large and growing number of factory

In [7]:
problems_and_ratings = [a + b for a, b in zip(problems_list, rated_problems_list)]

def print_menu(options):
    for idx, option in enumerate(options, start=1):
        print(f"{idx}. {option}")
        print("\n---\n")
        
def get_user_choice(num_options):
    while True:
        try:
            choice = int(input("Enter your choice (number): "))
            if 1 <= choice <= num_options:
                return choice
            else:
                print(f"Please enter a number between 1 and {num_options}.")
        except ValueError:
            print("Please enter a valid integer.")

def main():
    options = problems_and_ratings
    print_menu(options)
    choice = get_user_choice(len(options))
    print(f"You chose: {options[choice - 1]}")
    return choice

if __name__ == "__main__":
    chosen_index = main() -1
    chosen_problem = problems_list[chosen_index]
    print("Your choice is:\n" + chosen_problem)

1. "Title": Adaptive Energy Management

"Explanation": In the realm of factory automation, the integration of deep reinforcement learning (DRL) and advanced simulation technologies marks a significant leap forward. However, an emerging challenge lies in the optimization of energy consumption amidst these advancements. With factories increasingly relying on automated systems powered by sophisticated algorithms and simulations, the demand for energy is escalating, leading to higher operational costs and a larger carbon footprint. The problem intensifies as these technologies become more integrated into daily operations, requiring continuous power for complex computations and machinery operation. Moreover, the variability in energy prices and the push for sustainability demand a more dynamic approach to energy management. Traditional static energy usage strategies are becoming obsolete, unable to adapt to the fluctuating nature of energy markets or to leverage periods of low demand for co

Enter your choice (number): 1
You chose: "Title": Adaptive Energy Management

"Explanation": In the realm of factory automation, the integration of deep reinforcement learning (DRL) and advanced simulation technologies marks a significant leap forward. However, an emerging challenge lies in the optimization of energy consumption amidst these advancements. With factories increasingly relying on automated systems powered by sophisticated algorithms and simulations, the demand for energy is escalating, leading to higher operational costs and a larger carbon footprint. The problem intensifies as these technologies become more integrated into daily operations, requiring continuous power for complex computations and machinery operation. Moreover, the variability in energy prices and the push for sustainability demand a more dynamic approach to energy management. Traditional static energy usage strategies are becoming obsolete, unable to adapt to the fluctuating nature of energy markets or to

# Getting data from Mergeflow's API 
(for solution finding)

In [8]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(20) # first 20 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 19 documents.


In [12]:
#Summarize the content of the 19 documents to reduce text input in next task

import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")

HuLoop Automation, a company at the forefront of developing AI-powered intelligent automation solutions, has successfully completed a $5M seed funding round. This financial milestone was led by Moneta Ventures, and saw contributions from a variety of other investors including Rebellion Ventures, Growth Factory, Vibranium VC, Nurture Ventures, Insta VC, among others. The injection of capital is set to propel HuLoop's ambition to transform the automation sector by making it more accessible, efficient, and cost-effective. HuLoop's innovative approach has garnered the interest of a diverse client base, particularly within the financial services and retail/Consumer Packaged Goods (CPG) industries. This funding round marks a significant step forward in the company's journey to redefine automation for businesses seeking to leverage the latest in artificial intelligence technologies.

Source: HuLoop Automation Announcement.

---

HuLoop Automation, a company based in Auburn, CA, specializing i

Terabase Energy, a company at the forefront of digital and automation advancements for solar power plants, has successfully secured $25 million in a new round of financing to expedite the development and commercial scaling of its innovative Terafab™ construction system. This financial boost, spearheaded by Fifth Wall, a leading asset manager dedicated to enhancing and decarbonizing the built environment, also saw contributions from EDP Ventures and previous investors, indicating strong confidence in Terabase Energy's mission and technology. The Terafab™ system represents a significant leap forward in the construction of solar power plants, promising to automate and digitize the process, thereby revolutionizing the industry.

The investment is not just a financial endorsement but also a reflection of a shared vision among the investors and Terabase Energy. Anastasia Istratova from Fifth Wall highlighted the groundbreaking nature of automating solar plant construction and the alignment w

Worlds Enterprises, a company based in Dallas, Texas, has recently secured a significant financial boost for its innovative work in the realm of factory automation and digital transformation. The firm announced it successfully raised $21 million in a Series A1 funding round. This financial milestone is a testament to the confidence and interest from a diverse group of investors. The funding round was spearheaded by Moneta Ventures, accompanied by contributions from a mix of venture capital firms and corporate investors including Align Capital, Green Park & Golf Ventures, Chevron Technology Ventures, Piva Capital, PerotJain, and Capital Factory.

Under the leadership of CEO Dave Copps and President Chris Rohde, Worlds Enterprises is pioneering the development of a groundbreaking 4D infrastructure. This technological advancement is aimed at constructing the Industrial Metaverse, a concept that integrates artificial intelligence (AI) with real-world manufacturing and operational processes

CloudNC, a company at the forefront of revolutionizing precision manufacturing, recently announced a significant funding milestone, having raised $45 million in a Series B round. This funding round was led by Autodesk, a leader in software design and manufacturing, and saw participation from major entities such as Lockheed Martin and British Patient Capital, as well as contributions from existing investors Atomico, Episode 1 Ventures, and QVentures. The influx of capital is earmarked for the further development of CloudNC's pioneering AI technology for manufacturing, specifically its Software as a Service (SaaS) product designed to simplify the operation of CNC machines, making precision part manufacturing as easy as a single click for a wide range of customers from large corporations to SMEs and even individuals.

Located in London and announced on June 21, 2022, via PRNewswire, CloudNC's ambition is to make precision manufacturing autonomous, thereby enhancing its efficiency, reliabi

Linear Labs, a smart energy company based in Fort Worth, Texas, successfully secured $17 million in Series A funding to bolster its innovative electric motor production, specifically catering to the Light Electric Vehicle (LEV) industry. This financial milestone was achieved with the lead investment from THRC Investments and Folsom Point Equity, alongside contributions from a mix of new and existing investors including Lowercarbon Capital, Kindred Ventures, Saltwater Capital, Champion Hill Ventures, OzoneX Ventures, and Capital Factory.

The infusion of capital is earmarked for enhancing the company's manufacturing capabilities, which encompasses broadening manufacturing expertise, fortifying supply chain infrastructure and logistics, advancing automation and robotics engineering, and recruiting additional engineering talent. Under the leadership of founder and CEO Brad Hunstable, Linear Labs is at the forefront of launching its new electric motor and intelligence drive system, named t

In [11]:
Problem = chosen_problem

print(Problem)

"Title": Adaptive Energy Management

"Explanation": In the realm of factory automation, the integration of deep reinforcement learning (DRL) and advanced simulation technologies marks a significant leap forward. However, an emerging challenge lies in the optimization of energy consumption amidst these advancements. With factories increasingly relying on automated systems powered by sophisticated algorithms and simulations, the demand for energy is escalating, leading to higher operational costs and a larger carbon footprint. The problem intensifies as these technologies become more integrated into daily operations, requiring continuous power for complex computations and machinery operation. Moreover, the variability in energy prices and the push for sustainability demand a more dynamic approach to energy management. Traditional static energy usage strategies are becoming obsolete, unable to adapt to the fluctuating nature of energy markets or to leverage periods of low demand for cost 

# Solution Finding

In [13]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    The solution should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 3 sentences.
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

print("Dreamer Solutions:")    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print("Realist Solutions:")    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print("Critics Solutions:")    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (3 words max)
    “Solved Problem”: 1 or 2 sentence summary of the problem that is being solved 
    “Solution”: Provide a summary of the solution and how it works (in 4-5 sentences). Be dilligent on this task and make sure yo use the length limit. 
    “Impact”: How does the solution solve the specific problem (in 2-3 sentences)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution (in 4-5 sentences)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

 
print("Summary of Solutions:")   
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



Dreamer Solutions:
### 1) Quantum Energy Optimization

- **Solution Summary:** Utilizing quantum computing algorithms for real-time, ultra-efficient energy management in factory automation. This system predicts and adapts to energy consumption needs with unprecedented speed and accuracy, leveraging the principles of quantum mechanics. The Quantum Energy Optimization system continuously evolves, learning from data patterns to optimize energy use and reduce costs significantly.
  
- **Customer Journey:** Factory operations managers will interact with an intuitive dashboard that provides insights and recommendations based on quantum algorithm analyses. They can see real-time energy consumption forecasts, adjust production schedules for optimal energy use, and automatically shift operations to take advantage of lower energy rates or avoid peak pricing, all powered by the backend quantum computing infrastructure.
  
- **Novelty of the Solution:** Quantum computing's application in real-time

Critics Solutions:
### 1) Quantum Energy Optimization

**Title**: Quantum Energy Optimization

**Summary**: This solution leverages quantum computing to predict and adapt to energy needs in real-time, aiming for dynamic, precise, and cost-effective energy management. It promises unparalleled speed and accuracy in optimizing energy consumption through advanced computational capabilities.

**Evaluation criteria 2 details**: The reliance on quantum computing, still in its nascent stages for practical applications, presents significant risks regarding feasibility, scalability, and the current lack of a skilled workforce.

**Evaluation criteria 2**: 1

---

### 2) AI-Powered Microgrids

**Title**: AI-Powered Microgrids

**Summary**: By integrating AI with decentralized microgrids, this approach autonomously manages energy, optimizing its production, storage, and consumption. It learns from usage patterns to enhance efficiency and sustainability, aligning with modern energy management goals.

# Solution Evaluation

In [14]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. When evaluating the opportunities, you should take the perspective of a potential client of these solutions, active in the specific industry of {query}.
You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Task: Evaluate the provided solutions based on detailed criteria.

Input Text: {summary_solutions}

Instructions:

Extract Solutions: Begin by identifying all solutions within the input text. For each solution, note the following details:
    - Name
    - Solved Problem
    - Solution Description
    - Impact
    - Technology Used
    - Sources
    
Develop Evaluation Criteria: Create 10 specific criteria to assess the solutions. These criteria should enable a clear comparison between the solutions, focusing on aspects such as innovation, scalability, social impact, and cost-effectiveness.

Rate Each Solution: Apply the criteria to each solution, assigning ratings on a scale from 1 to 3 (1=Lowest, 3=Highest). Be stringent with the ratings:
    1: Does not meet expectations
    2: Meets expectations adequately
    3: Exceeds expectations significantly

A rating of 3 should be reserved for exceptional performance in the given criterion.

Then, also output the total score of a solution, out of the maximum of 30 points.

Repeat for All Solutions: Follow the steps above for each solution presented in the input text.

Objective: This structured analysis will facilitate a comprehensive comparison of the solutions, highlighting their strengths and areas for improvement based on the defined criteria.

Create a table which gives an overview of the solutions and their ratings in the different criteria, giving a number to each solution and only mentionning the number in the horizontal axis of the table. The solution name should not be included in the table.

In your output, please ensure that each solution is separated by "\n---\n".
"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



### Extracted Solutions

1. **Quantum Energy Management**
   - **Solved Problem**: Real-time, ultra-efficient energy management in automated factories.
   - **Solution Description**: Uses quantum computing to dynamically predict and adapt to energy needs, learning from data patterns to optimize energy use.
   - **Impact**: Reduces operational costs and carbon footprint, aligns with sustainability goals.
   - **Technology Used**: Quantum computing, specialized algorithms for energy management, integration with factory systems.
   - **Sources**: HuLoop Automation, Terabase Energy.

2. **AI-Powered Microgrids**
   - **Solved Problem**: Dynamic management of decentralized, factory-based microgrids integrating renewable energy.
   - **Solution Description**: AI algorithms manage microgrid assets, optimizing energy production, storage, and consumption.
   - **Impact**: Enhances energy efficiency, reduces reliance on traditional power grids.
   - **Technology Used**: AI algorithms, renewable 

In [15]:
# Pick the best three 

user_prompt5 = f"""
Extract and rank the solutions based on their overall ratings from the provided data: {initial_rating}.

Identify the top three solutions with the highest overall ratings out of 30 and sort them in descending order of their ratings.

For each of these top solutions, provide detailed information based on the content from: {summary_solutions}. Ensure comprehensive coverage of the points listed below:

- "Name": The solution's name, limited to a maximum of 5 words.
- "Solved Problem": A concise three-sentence summary of the problem addressed by the solution (minimum 50 words).
- "Solution": A brief summary of the solution, including its mechanism, in 4 to 5 sentences.
- "Impact": Explanation of how the solution addresses the specific problem, in 2 to 3 sentences.
- "Technology": Detailed description of the technologies used, including their application, aimed at a CTO with an engineering background for implementation purposes, in 4-5 sentences.
- "Sources": Mention any specific inspiration sources from {summary_list}, limited to 1 sentence.

Additionally, highlight the rating criteria where each solution excels.

Please separate each solution's details with "---".

"""

# Call the GPT model to generate the completion
completion = openai.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
    {"role": "system", "content": system_prompt_2},
    {"role": "user", "content": user_prompt5}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

# Correctly access the generated text from the completion response
generated_text = completion.choices[0].message.content

# Split the generated text into individual solutions based on your specified separator ("---")
solutions_list = generated_text.split("---")

# Initialize an empty dictionary to hold your final solutions
final_solutions = {}

# Populate the dictionary with distinct solutions
for index, solution_text in enumerate(solutions_list, start=1):
    if solution_text.strip():  # Ensure the string is not just whitespace
        final_solutions[index] = solution_text.strip()

# Ensure only the top 3 solutions are kept if more were accidentally generated
final_solutions = dict(list(final_solutions.items())[:3])

# 'final_solutions' now contains up to three distinct items as requested

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


### AI-Powered Microgrids
- **Name**: AI Microgrid Control
- **Solved Problem**: This solution tackles the challenge of managing decentralized, factory-based microgrids with renewable energy integration, optimizing energy production, storage, and consumption for enhanced efficiency and sustainability.
- **Solution**: AI Microgrid Control uses advanced AI algorithms to autonomously manage and optimize microgrid assets such as solar panels and wind turbines, predicting energy patterns and adjusting usage and storage accordingly.
- **Impact**: By enhancing energy efficiency and reducing reliance on traditional power grids, this solution aligns with sustainability targets and operational cost reduction, offering a dynamic and efficient energy management approach for factories.
- **Technology**: The core technology includes AI algorithms for energy management, integration with renewable energy sources, and energy storage systems, alongside a data analytics platform for pattern recognition a

# Integrate further Mergeflow Data

In [16]:
# Integrate new mergeflow data for the solutions 

# Now, based on soultions found, we need more info from Mergeflow


boolean_queries = {}

for index, solution_name in final_solutions.items():
    user_prompt6 = f"""
    
    Create a boolean query based on the Name of the solution from {solution_name}.
    
    Please use the following as an example and follow the format: 
    
    Example: for Factory Blockchain Protocol, this would be:

    "factory blockchain protocol" OR "blockchain for manufacturing" OR "blockchain in production" OR "blockchain in factories" OR "smart factory blockchain" OR "industrial blockchain" OR "supply chain blockchain" OR "blockchain logistics" OR "blockchain traceability"
    

    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
        {"role": "system", "content": system_prompt_2},
        {"role": "user", "content": user_prompt6}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )

    # Extracting the content of the message from the first choice in the choices list
    query_content = completion.choices[0].message.content
    
    # Save the generated boolean query for the current solution
    boolean_queries[index] = query_content

    # Print the generated boolean query for the current solution
    print(query_content)
    print("\n----\n")
    

    

"AI microgrid control" OR "microgrid optimization" OR "renewable energy management in factories" OR "AI for energy efficiency in manufacturing" OR "smart microgrids in industrial settings" OR "adaptive energy management systems" OR "AI-driven energy optimization" OR "sustainable energy solutions for factories" OR "energy storage and distribution AI" OR "intelligent microgrid management" OR "factory energy sustainability" OR "renewable integration in factory operations"

----

"smart energy trading" OR "predictive energy trading" OR "energy cost optimization" OR "renewable energy trading" OR "automated energy trading" OR "dynamic energy management" OR "energy trading blockchain" OR "DRL energy forecasting" OR "blockchain energy transactions" OR "sustainability energy management"

----

"waste-to-energy bioreactors" OR "bioreactor for waste management" OR "clean energy from waste" OR "genetically engineered microorganisms for energy" OR "biotechnological waste-to-energy solutions" OR "su

In [18]:
dataset = '&sp=3814'  # access dataset from mergeflow - in this case industry news
rows = '3'  # first 3 docs only

# Iterate through each boolean query
for solution, query_text in boolean_queries.items():
    # Correctly encode the query
    encoded_query = query_text.replace(" OR ", '%7C').replace('" ', '%22').replace(' "', '%22').replace(' ', '+').replace('"', '%22')

    # Construct the API request URL
    url = f'https://mergeflow.net/api/v1/{username}/search/getContent?q={encoded_query}{dataset}&rows={rows}'

    mergeflow_url = f'https://mergeflow.net/Profiles/{username}/search?date=default3&q={encoded_query}{dataset}&rows={rows}'

    
    # Define the headers to be sent with the request
    headers = {
        'MergeflowNet-Api-Auth-Key': mergeflow_api_key
    }

    # Send the POST request
    response = requests.post(url, headers=headers)

    mergeflow_docs = []

    # Check the response status code and content
    if response.status_code == 200:
        mergeflow_response_json = response.json()
        
        # Extract and store documents information
        for document in mergeflow_response_json['Documents']:
            current_doc = {
                'title': document['Title'],
                'date': document['Date'],
                'url': document['Url'],
                'content': document['Content']
            }
            
            mergeflow_docs.append(current_doc)
            


    # Process or print your results here
    # For example, print titles of documents for this query
    print(f"Results for Solution {solution}:")
    print("\n-\n") 
    print(f"Mergeflow URL:")
    print(mergeflow_url)
    print("\n-\n") 

    for doc in mergeflow_docs:
        print(doc['title'])
        print(doc['url'])
    print("\n----\n")
    

    

Results for Solution 1:

-

Mergeflow URL:
https://mergeflow.net/Profiles/leonard/search?date=default3&q=%22AI+microgrid+control%22%7C%22microgrid+optimization%22%7C%22renewable+energy+management+in+factories%22%7C%22AI+for+energy+efficiency+in+manufacturing%22%7C%22smart+microgrids+in+industrial+settings%22%7C%22adaptive+energy+management+systems%22%7C%22AI-driven+energy+optimization%22%7C%22sustainable+energy+solutions+for+factories%22%7C%22energy+storage+and+distribution+AI%22%7C%22intelligent+microgrid+management%22%7C%22factory+energy+sustainability%22%7C%22renewable+integration+in+factory+operations%22&sp=3814&rows=3

-

Schneider Electric - Focus on Hydrogen Safety Aligns with Growing Economy - Hydrogen Central
https://hydrogen-central.com/schneider-electric-focus-hydrogen-safety-aligns-growing-economy/
XENDEE Releases New Generative Algorithm to Enable Multi-Year Microgrid Optimization with Industry Leading Accuracy and 12,000% Runtime Reduction | Business Wire
http://www.busin